# 実験結果の可視化

In [1]:
%load_ext autoreload
%autoreload 2

import os
import configparser
import sys

import numpy as np
import pandas as pd
# import torch
import plotly.graph_objects as go

# 設定の読み込み
config = configparser.ConfigParser()
config_path = "../config.ini"
config.read(config_path)
PROJECT_DIR = config["paths"]["project_dir"]
LOG_DIR = config["paths"]["logs_dir"]
sys.path.append(PROJECT_DIR)

from src.utils_experiment import OptimLogParser_v1
from src.utils_experiment import OptimLogParser
from src.utils_experiment import search_log_files

# 08-29

In [2]:
search_log_files(LOG_DIR, ["2024-08-29"])

['2024-08-29_16-16-41_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-19-08_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-21-58_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-26-55_Simple_5d_unconstrained_tanh_2.log',
 '2024-08-29_16-29-52_Simple_5d_unconstrained_tanh_2.log',
 '2024-08-29_16-30-07_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-31-12_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-32-22_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-41-26_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-42-15_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-44-02_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-48-54_Simple_5d_constrained3_tanh_3.log',
 '2024-08-29_16-59-16_Simple_5d_constrained3_tanh_4.log',
 '2024-08-29_17-02-03_Simple_5d_constrained3_tanh_5.log']

### visualization for constrained bo

In [1]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from src.utils_experiment import OptimLogParser


# Function to parse the log file and return the merged DataFrame
def parse_and_prepare_data(log_file_path):
    parser = OptimLogParser(log_file_path)
    parser.parse_log_file()

    settings = parser.settings
    initial_data = parser.initial_data
    bo_data = parser.bo_data

    columns = [
        "iteration",
        "iteration_time",
        "candidate",
        "function_value",
        "best_f",
        "acquisition_value",
        "surrogate_mean",
        "surrogate_covariance",
        "final_training_loss",
    ]

    df_merged = pd.concat([initial_data, bo_data], axis=0)
    df_merged["best_f"] = df_merged["function_value"].cummax()
    df_merged = df_merged[columns]
    df_merged["iteration"] = df_merged["iteration"].fillna(0).astype(int)

    # Apply constraints
    df_merged["candidate"] = df_merged["candidate"].apply(restore_array)
    df_merged["constraint"] = df_merged["candidate"].apply(g)

    # Calculate best_f_constrained, considering only rows where constraint is True
    df_merged["best_f_constrained"] = df_merged.apply(
        lambda row: row["function_value"] if row["constraint"] else float("-inf"),
        axis=1,
    ).cummax()

    # Replace negative infinity with NaN for rows where constraint is False
    df_merged["best_f_constrained"] = df_merged["best_f_constrained"].replace(
        float("-inf"), np.nan
    )

    return df_merged, settings


# Function to restore array from string representation
def restore_array(str_array):
    str_array = str_array.strip("'").strip("[]")
    array = np.fromstring(str_array, sep=" ")
    return array.reshape(1, -1)


# Constraint function
def g(X):
    """
    Constraint: x1 == x2
    """
    X1 = X[:, 0]
    X2 = X[:, 1]

    return (X1 == X2)[0]


# Function to plot the data
def plot_function_values(df_merged):
    fig = go.Figure()

    # Add scatter trace for function_value with color based on constraint
    fig.add_trace(
        go.Scatter(
            x=df_merged["iteration"],
            y=df_merged["function_value"],
            mode="markers",
            marker=dict(
                color=df_merged["constraint"].apply(lambda x: "orange" if x else "blue")
            ),
            name="Function Value",
        )
    )

    # Add line trace for best_f
    fig.add_trace(
        go.Scatter(
            x=df_merged["iteration"],
            y=df_merged["best_f"],
            mode="lines",
            line=dict(color="red"),
            name="Best f",
        )
    )

    # Add line trace for best_f_constrained
    fig.add_trace(
        go.Scatter(
            x=df_merged["iteration"],
            y=df_merged["best_f_constrained"],
            mode="lines",
            line=dict(color="green"),
            name="Best f under constraint",
        )
    )

    # Update layout
    fig.update_layout(
        title="Function Value over Iterations with Constraints",
        xaxis_title="Iteration",
        yaxis_title="Function Value",
        showlegend=True,
    )

    # Show the figure
    fig.show()


# # Example usage ------------------------------------------------------------------------------
# log_file_path = os.path.join(
#     LOG_DIR, "2024-08-27_20-12-18_Simple_5d_constrained_normalized-input_2.log"
# )
# df_merged, settings = parse_and_prepare_data(log_file_path)

# print("実験設定")
# display(settings)
# print()
# display(df_merged)

# plot_function_values(df_merged)


ModuleNotFoundError: No module named 'src'

In [7]:
search_log_files(LOG_DIR, ["2024-08-29"])

['2024-08-29_16-16-41_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-19-08_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-21-58_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-26-55_Simple_5d_unconstrained_tanh_2.log',
 '2024-08-29_16-29-52_Simple_5d_unconstrained_tanh_2.log',
 '2024-08-29_16-30-07_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-31-12_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-32-22_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-41-26_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-42-15_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-44-02_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-48-54_Simple_5d_constrained3_tanh_3.log',
 '2024-08-29_16-59-16_Simple_5d_constrained3_tanh_4.log',
 '2024-08-29_17-02-03_Simple_5d_constrained3_tanh_5.log']

In [10]:
search_log_files(LOG_DIR, ["2024-08-29", "tanh_3"])

['2024-08-29_16-41-26_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-42-15_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-44-02_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-48-54_Simple_5d_constrained3_tanh_3.log']

#### lambda1=0.5, lambda2=0.5, clipping=False

- 獲得関数の符号を間違えている

In [9]:
filename = search_log_files(LOG_DIR, ["2024-08-29", "tanh_3"])[2]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-29_16-44-02_Simple_5d_unconstrained_tanh_3.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_3',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[5.0, -2.0, 3.0, 9.0, 1.0]]",-120.0,-120.0,NaN,NaN,NaN,92.931175,False,NaN
1,0,NaN,"[[-4.0, 0.0, -4.0, 4.0, 5.0]]",-73.0,-73.0,NaN,NaN,NaN,92.931175,False,NaN
2,0,NaN,"[[7.0, 10.0, 0.0, -4.0, -2.0]]",-169.0,-73.0,NaN,NaN,NaN,92.931175,False,NaN
3,0,NaN,"[[10.0, -2.0, 9.0, 1.0, -9.0]]",-267.0,-73.0,NaN,NaN,NaN,92.931175,False,NaN
4,0,NaN,"[[-7.0, 7.0, -4.0, 10.0, 2.0]]",-218.0,-73.0,NaN,NaN,NaN,92.931175,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
998,999,0.6667,"[[-6.0, -3.0, -5.0, 6.0, -1.0]]",-107.0,-26.0,-72.514160,-65.329132,12085.382812,7.468862,False,-26.0
999,1000,0.6684,"[[2.0, -1.0, -5.0, -2.0, -8.0]]",-98.0,-26.0,-118.985893,-53.838482,12163.668945,7.217340,False,-26.0
1000,1001,0.7379,"[[5.0, -8.0, -3.0, 1.0, 7.0]]",-148.0,-26.0,-124.859154,-167.716034,10682.611328,6.291318,False,-26.0
1001,1002,0.6880,"[[-9.0, -0.0, 10.0, 10.0, -8.0]]",-345.0,-26.0,-79.105042,-287.606262,11881.660156,6.417652,False,-26.0


In [13]:
search_log_files(LOG_DIR, ["2024-08-29"])

['2024-08-29_16-16-41_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-19-08_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-21-58_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-26-55_Simple_5d_unconstrained_tanh_2.log',
 '2024-08-29_16-29-52_Simple_5d_unconstrained_tanh_2.log',
 '2024-08-29_16-30-07_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-31-12_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-32-22_Simple_5d_unconstrained_tanh.log',
 '2024-08-29_16-41-26_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-42-15_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-44-02_Simple_5d_unconstrained_tanh_3.log',
 '2024-08-29_16-48-54_Simple_5d_constrained3_tanh_3.log',
 '2024-08-29_16-59-16_Simple_5d_constrained3_tanh_4.log',
 '2024-08-29_17-02-03_Simple_5d_constrained3_tanh_5.log']

In [15]:
filename = search_log_files(LOG_DIR, ["2024-08-29", "tanh_4"])[0]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-29_16-59-16_Simple_5d_constrained3_tanh_4.log

実験設定


{'name': 'Simple_5d_constrained3_tanh_4',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[0.0, -1.0, -4.0, 8.0, -6.0]]",-117.0,-117.0,NaN,NaN,NaN,0.000000,False,NaN
1,0,NaN,"[[4.0, -2.0, -4.0, 4.0, 4.0]]",-68.0,-68.0,NaN,NaN,NaN,0.000000,False,NaN
2,0,NaN,"[[-4.0, 3.0, 0.0, 7.0, -3.0]]",-83.0,-68.0,NaN,NaN,NaN,0.000000,False,NaN
3,0,NaN,"[[-5.0, -2.0, 7.0, -6.0, -9.0]]",-195.0,-68.0,NaN,NaN,NaN,0.000000,False,NaN
4,0,NaN,"[[-6.0, -7.0, -9.0, -9.0, -6.0]]",-283.0,-68.0,NaN,NaN,NaN,0.000000,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1010,1011,1.4614,"[[-7.0, 3.0, -9.0, 1.0, 1.0]]",-141.0,-14.0,-91.263069,-166.253952,5648.978027,197.405624,False,-43.0
1011,1012,1.4660,"[[-10.0, -4.0, -1.0, -10.0, 8.0]]",-281.0,-14.0,-70.504822,-197.658844,5928.669922,189.852737,False,-43.0
1012,1013,1.4368,"[[-7.0, 3.0, -3.0, -7.0, 9.0]]",-197.0,-14.0,-106.809502,-82.302170,5979.110352,212.425903,False,-43.0
1013,1014,1.5289,"[[8.0, 4.0, 8.0, -5.0, 5.0]]",-194.0,-14.0,-118.493729,-220.658691,5958.570801,187.845734,False,-43.0


In [14]:
filename = search_log_files(LOG_DIR, ["2024-08-29", "tanh_3"])[3]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-29_16-48-54_Simple_5d_constrained3_tanh_3.log

実験設定


{'name': 'Simple_5d_constrained3_tanh_3',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[4.0, 1.0, -4.0, 1.0, 4.0]]",-50.0,-50.0,NaN,NaN,NaN,14.690071,False,NaN
1,0,NaN,"[[0.0, 8.0, -7.0, -5.0, -8.0]]",-202.0,-50.0,NaN,NaN,NaN,14.690071,False,NaN
2,0,NaN,"[[-4.0, -5.0, -5.0, 4.0, -9.0]]",-163.0,-50.0,NaN,NaN,NaN,14.690071,False,NaN
3,0,NaN,"[[-10.0, -10.0, 8.0, 8.0, -2.0]]",-332.0,-50.0,NaN,NaN,NaN,14.690071,True,-332.0
4,0,NaN,"[[0.0, 4.0, 8.0, -2.0, -6.0]]",-120.0,-50.0,NaN,NaN,NaN,14.690071,False,-332.0
...,...,...,...,...,...,...,...,...,...,...,...
560,561,41.7615,"[[-5.0, -3.0, -1.0, -10.0, -9.0]]",-216.0,-14.0,-34783.484375,-35334.179688,20966512.0,-14219.564453,False,-46.0
561,562,39.4026,"[[7.0, 4.0, 8.0, 5.0, -9.0]]",-235.0,-14.0,-36487.203125,-32730.759766,21370882.0,-14275.141602,False,-46.0
562,563,37.9320,"[[-5.0, -3.0, -10.0, 7.0, -6.0]]",-219.0,-14.0,-26325.888672,-31389.394531,20462056.0,-14329.471680,False,-46.0
563,564,37.8537,"[[-7.0, -9.0, 8.0, 8.0, -8.0]]",-322.0,-14.0,-4974.633301,-20761.792969,20729396.0,-14464.359375,False,-46.0


In [15]:
filename = search_log_files(LOG_DIR, ["2024-08-29", "tanh_5"])[0]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-29_17-02-03_Simple_5d_constrained3_tanh_5.log

実験設定


{'name': 'Simple_5d_constrained3_tanh_5',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-2.0, 6.0, -1.0, 7.0, 3.0]]",-99.0,-99.0,NaN,NaN,NaN,-7.569180,False,NaN
1,0,NaN,"[[0.0, 1.0, -3.0, 9.0, -7.0]]",-140.0,-99.0,NaN,NaN,NaN,-7.569180,False,NaN
2,0,NaN,"[[0.0, -10.0, 0.0, -3.0, -1.0]]",-110.0,-99.0,NaN,NaN,NaN,-7.569180,False,NaN
3,0,NaN,"[[-2.0, -1.0, 0.0, -7.0, 2.0]]",-58.0,-58.0,NaN,NaN,NaN,-7.569180,False,NaN
4,0,NaN,"[[-10.0, 0.0, 1.0, 0.0, 5.0]]",-126.0,-58.0,NaN,NaN,NaN,-7.569180,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
541,542,40.8588,"[[-4.0, -1.0, -8.0, 9.0, 10.0]]",-262.0,-31.0,-12495.496094,-12326.174805,14521.811523,114.563126,False,-44.0
542,543,36.6768,"[[-9.0, -8.0, 9.0, 3.0, 7.0]]",-284.0,-31.0,-218.256790,-12512.034180,14496.779297,131.314850,False,-44.0
543,544,36.1125,"[[4.0, 4.0, 9.0, -9.0, 9.0]]",-275.0,-31.0,-417.420929,-665.334839,14377.213867,344.361420,True,-44.0
544,545,36.2904,"[[-9.0, -3.0, 9.0, 5.0, 6.0]]",-232.0,-31.0,-12282.157227,-12355.106445,14299.658203,203.806686,False,-44.0


In [3]:
filename = search_log_files(LOG_DIR, ["2024-08-29", "tanh_6"])[0]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-29_23-26-14_Simple_5d_unconstrained_tanh_6.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_6',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 256,
  'num_hidden_layers': 4,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[7.0, -2.0, -2.0, 0.0, 8.0]]",-121.0,-121.0,NaN,NaN,NaN,13.458967,False,NaN
1,0,NaN,"[[-7.0, -6.0, 0.0, -7.0, 4.0]]",-150.0,-121.0,NaN,NaN,NaN,13.458967,False,NaN
2,0,NaN,"[[-6.0, 0.0, 8.0, 9.0, 6.0]]",-217.0,-121.0,NaN,NaN,NaN,13.458967,False,NaN
3,0,NaN,"[[0.0, 2.0, 5.0, 5.0, -4.0]]",-70.0,-70.0,NaN,NaN,NaN,13.458967,False,NaN
4,0,NaN,"[[-2.0, -6.0, 7.0, -3.0, 3.0]]",-107.0,-70.0,NaN,NaN,NaN,13.458967,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
811,812,0.7359,"[[10.0, -10.0, -10.0, -10.0, 10.0]]",-500.0,-63.0,552.340149,516.090637,2816.193604,5.367833,False,-337.0
812,813,0.7506,"[[10.0, -10.0, -10.0, -10.0, 10.0]]",-500.0,-63.0,408.896637,394.949768,2721.213135,5.418570,False,-337.0
813,814,0.7232,"[[10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-63.0,493.880524,629.057434,2774.397217,5.377799,False,-337.0
814,815,0.7312,"[[-10.0, -10.0, 10.0, -10.0, 10.0]]",-500.0,-63.0,377.085938,493.336975,2778.001709,6.414770,True,-337.0


In [4]:
filename = search_log_files(LOG_DIR, ["2024-08-29", "tanh_6"])[1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-29_23-34-21_Simple_5d_unconstrained_tanh_6.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_6',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 256,
  'num_hidden_layers': 4,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[3.0, 2.0, -10.0, 4.0, -5.0]]",-154.0,-154.0,NaN,NaN,NaN,33.768463,False,NaN
1,0,NaN,"[[-5.0, -9.0, 10.0, 1.0, 1.0]]",-208.0,-154.0,NaN,NaN,NaN,33.768463,False,NaN
2,0,NaN,"[[0.0, 0.0, -9.0, -5.0, 3.0]]",-115.0,-115.0,NaN,NaN,NaN,33.768463,True,-115.0
3,0,NaN,"[[-9.0, -7.0, 1.0, -1.0, -1.0]]",-133.0,-115.0,NaN,NaN,NaN,33.768463,False,-115.0
4,0,NaN,"[[-2.0, 7.0, -1.0, -6.0, 8.0]]",-154.0,-115.0,NaN,NaN,NaN,33.768463,False,-115.0
...,...,...,...,...,...,...,...,...,...,...,...
280,281,0.2375,"[[6.0, 5.0, 0.0, 5.0, 3.0]]",-95.0,-26.0,-80.645782,-324.900330,18759.197266,6.796646,False,-38.0
281,282,0.2476,"[[10.0, -10.0, -10.0, -10.0, 10.0]]",-500.0,-26.0,-130.433243,-155.279861,18850.925781,6.598773,False,-38.0
282,283,0.2703,"[[-10.0, 10.0, -10.0, -10.0, 10.0]]",-500.0,-26.0,77.948425,-149.192825,18519.992188,6.849128,False,-38.0
283,284,0.2340,"[[3.0, 9.0, 4.0, -8.0, -2.0]]",-174.0,-26.0,-82.653397,-382.281586,19245.693359,7.628325,False,-38.0


In [5]:
filename = search_log_files(LOG_DIR, ["2024-08-29", "tanh_6"])[2]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-29_23-36-42_Simple_5d_unconstrained_tanh_6.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_6',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 256,
  'num_hidden_layers': 4,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[5.0, 10.0, -7.0, -2.0, 9.0]]",-259.0,-259.0,NaN,NaN,NaN,10.024561,False,NaN
1,0,NaN,"[[-3.0, 5.0, -2.0, -4.0, 0.0]]",-54.0,-54.0,NaN,NaN,NaN,10.024561,False,NaN
2,0,NaN,"[[8.0, -6.0, 2.0, 10.0, -5.0]]",-229.0,-54.0,NaN,NaN,NaN,10.024561,False,NaN
3,0,NaN,"[[1.0, 6.0, 1.0, 1.0, -8.0]]",-103.0,-54.0,NaN,NaN,NaN,10.024561,False,NaN
4,0,NaN,"[[-1.0, 2.0, -2.0, 6.0, -8.0]]",-109.0,-54.0,NaN,NaN,NaN,10.024561,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
157,158,0.1457,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-53.0,336.426575,404.351074,9244.792969,8.496299,True,-96.0
158,159,0.1491,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-53.0,390.878479,447.344879,8990.369141,6.237096,True,-96.0
159,160,0.1472,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-53.0,496.538757,560.821350,8763.716797,6.056902,True,-96.0
160,161,0.1484,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-53.0,342.642242,583.549683,8890.850586,6.509725,True,-96.0


In [6]:
search_log_files(LOG_DIR, ["2024-08-29", "tanh_6"])

['2024-08-29_23-26-14_Simple_5d_unconstrained_tanh_6.log',
 '2024-08-29_23-34-21_Simple_5d_unconstrained_tanh_6.log',
 '2024-08-29_23-36-42_Simple_5d_unconstrained_tanh_6.log',
 '2024-08-29_23-49-11_Simple_5d_unconstrained_tanh_6.log',
 '2024-08-29_23-52-14_Simple_5d_unconstrained_tanh_6.log',
 '2024-08-29_23-53-37_Simple_5d_unconstrained_tanh_6.log',
 '2024-08-29_23-58-59_Simple_5d_unconstrained_tanh_6.log']

In [7]:
filename = search_log_files(LOG_DIR, ["2024-08-29", "tanh_6"])[-1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-29_23-58-59_Simple_5d_unconstrained_tanh_6.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_6',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 256,
  'num_hidden_layers': 4,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[0.0, 1.0, 1.0, 6.0, 3.0]]",-47.0,-47.0,NaN,NaN,NaN,23.540058,False,NaN
1,0,NaN,"[[5.0, -1.0, 9.0, -3.0, -3.0]]",-125.0,-47.0,NaN,NaN,NaN,23.540058,False,NaN
2,0,NaN,"[[-6.0, 10.0, -4.0, -10.0, 2.0]]",-256.0,-47.0,NaN,NaN,NaN,23.540058,False,NaN
3,0,NaN,"[[-3.0, -2.0, 6.0, -10.0, -9.0]]",-230.0,-47.0,NaN,NaN,NaN,23.540058,False,NaN
4,0,NaN,"[[9.0, 3.0, 8.0, 0.0, -8.0]]",-218.0,-47.0,NaN,NaN,NaN,23.540058,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
70,71,0.1165,"[[3.0, 2.0, 4.0, -7.0, -1.0]]",-79.0,-31.0,-98.022865,-308.707062,11649.171875,6.167607,False,-31.0
71,72,0.1157,"[[-7.0, 4.0, -10.0, 3.0, -8.0]]",-238.0,-31.0,-116.024094,-276.418823,11775.089844,6.905889,False,-31.0
72,73,0.1267,"[[-1.0, -8.0, -1.0, -7.0, -3.0]]",-124.0,-31.0,-76.730423,-72.190125,11408.015625,6.084595,False,-31.0
73,74,0.1160,"[[10.0, 8.0, -6.0, -6.0, 7.0]]",-285.0,-31.0,90.765549,-64.478767,11448.059570,6.229772,False,-31.0


In [9]:
filename = search_log_files(LOG_DIR, ["2024-08-30", "tanh_6"])[-1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-30_00-19-33_Simple_5d_unconstrained_tanh_6.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_6',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 10,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 1000, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-8.0, -5.0, 7.0, -5.0, -1.0]]",-164.0,-164.0,NaN,NaN,NaN,6.218917,False,NaN
1,0,NaN,"[[-7.0, -1.0, 9.0, 5.0, -9.0]]",-237.0,-164.0,NaN,NaN,NaN,6.218917,False,NaN
2,0,NaN,"[[-4.0, 1.0, 10.0, -4.0, -5.0]]",-158.0,-158.0,NaN,NaN,NaN,6.218917,False,NaN
3,0,NaN,"[[7.0, -8.0, -2.0, 4.0, 10.0]]",-233.0,-158.0,NaN,NaN,NaN,6.218917,False,NaN
4,0,NaN,"[[-8.0, 2.0, 10.0, 2.0, 4.0]]",-188.0,-158.0,NaN,NaN,NaN,6.218917,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
280,281,1.1171,"[[-0.0, -2.0, -5.0, -9.0, -6.0]]",-146.0,-18.0,177.034988,-163.504028,9076.931641,6.019305,False,-18.0
281,282,1.1115,"[[-9.0, -5.0, -6.0, 8.0, 2.0]]",-210.0,-18.0,346.287415,-256.310608,8912.878906,6.007405,False,-18.0
282,283,1.1838,"[[7.0, -9.0, 6.0, -6.0, -5.0]]",-227.0,-18.0,371.679810,-219.971756,9076.874023,6.115738,False,-18.0
283,284,1.1499,"[[-2.0, -5.0, -2.0, -5.0, -3.0]]",-67.0,-18.0,339.580566,-190.294434,8717.679688,6.194726,False,-18.0


In [10]:
filename = search_log_files(LOG_DIR, ["2024-08-30", "tanh_6"])[-1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-30_00-25-05_Simple_5d_unconstrained_tanh_6.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_6',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 1000, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[2.0, -5.0, 3.0, 3.0, 1.0]]",-48.0,-48.0,NaN,NaN,NaN,3.000283,False,NaN
0,1,0.5200,"[[2.0, -10.0, -4.0, -8.0, -8.0]]",-248.0,-48.0,54.417858,-54.501648,45.879539,6.838418,False,NaN
1,2,0.5287,"[[-8.0, 4.0, -5.0, -0.0, -4.0]]",-121.0,-48.0,110.630547,-53.378132,6712.688965,6.340144,False,NaN
2,3,0.5316,"[[-8.0, -5.0, -8.0, 4.0, -2.0]]",-173.0,-48.0,288.269928,-134.568100,6689.098633,6.484483,False,NaN
3,4,0.5424,"[[-8.0, 1.0, 0.0, -9.0, -8.0]]",-210.0,-48.0,190.394104,-91.778770,5718.464355,6.164846,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
141,142,0.9470,"[[-6.0, 5.0, 6.0, -4.0, 1.0]]",-114.0,-38.0,142.298691,8.657777,9783.294922,6.663619,False,-73.0
142,143,0.9303,"[[-9.0, -4.0, -6.0, 1.0, -4.0]]",-150.0,-38.0,247.527252,-122.793381,8743.204102,7.300663,False,-73.0
143,144,0.9086,"[[-8.0, 10.0, -8.0, -7.0, 3.0]]",-286.0,-38.0,134.758133,-335.421265,9308.934570,7.483754,False,-73.0
144,145,0.8775,"[[4.0, 1.0, -9.0, -7.0, 6.0]]",-183.0,-38.0,233.793854,-214.807327,8423.719727,7.041640,False,-73.0


In [11]:
filename = search_log_files(LOG_DIR, ["2024-08-30", "tanh_6"])[-1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-30_00-27-44_Simple_5d_unconstrained_tanh_6.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_6',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.01},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-5.0, -2.0, -1.0, 4.0, 7.0]]",-95.0,-95.0,NaN,NaN,NaN,7.444461,False,NaN
0,1,0.0651,"[[-3.0, -5.0, -7.0, -2.0, -4.0]]",-103.0,-95.0,42.984818,-26.329952,135.263962,3.758239,False,NaN
1,2,0.0606,"[[-7.0, -4.0, 10.0, -8.0, -1.0]]",-230.0,-95.0,94.599533,-62.156971,212.830734,8.279942,False,NaN
2,3,0.0624,"[[-1.0, -5.0, 2.0, 3.0, -9.0]]",-120.0,-95.0,93.074516,-108.223862,575.977966,6.204121,False,NaN
3,4,0.0586,"[[-7.0, 5.0, -8.0, -9.0, -0.0]]",-219.0,-95.0,148.291260,-188.254654,1147.440674,5.706096,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
360,361,0.1283,"[[7.0, -8.0, -9.0, -4.0, -4.0]]",-226.0,-13.0,312.304108,-176.057098,6269.667969,5.820759,False,-13.0
361,362,0.1282,"[[-2.0, -9.0, -6.0, 4.0, -3.0]]",-146.0,-13.0,59.983345,-167.177643,6302.945312,5.792138,False,-13.0
362,363,0.1263,"[[-1.0, -8.0, -4.0, 1.0, -4.0]]",-98.0,-13.0,234.927811,-64.137238,6216.170898,6.445852,False,-13.0
363,364,0.1233,"[[7.0, -9.0, -7.0, -1.0, -8.0]]",-244.0,-13.0,121.052353,-116.030273,6256.095215,5.836024,False,-13.0


In [12]:
filename = search_log_files(LOG_DIR, ["2024-08-30", "tanh_6"])[-1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged)

plot_function_values(df_merged)

2024-08-30_02-08-03_Simple_5d_unconstrained_tanh_6.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh_6',
 'device': 'cpu',
 'bo_iter': 10000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-4.0, 2.0, -3.0, -1.0, -2.0]]",-34.0,-34.0,NaN,NaN,NaN,6.617652,False,NaN
0,1,0.0613,"[[-8.0, -3.0, -5.0, -10.0, -10.0]]",-298.0,-34.0,2.226378,2.824693,41.238678,242.850708,False,NaN
1,2,0.0638,"[[-2.0, -9.0, -7.0, -3.0, -2.0]]",-147.0,-34.0,26.706583,-0.766337,82.982018,161.666245,False,NaN
2,3,0.0576,"[[-4.0, -8.0, 3.0, -10.0, -9.0]]",-270.0,-34.0,19.156391,-22.607119,100.051468,152.591110,False,NaN
3,4,0.0611,"[[-4.0, -2.0, -9.0, 4.0, -6.0]]",-153.0,-34.0,19.339819,-38.695847,120.062531,131.945770,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
451,452,0.1387,"[[-10.0, 1.0, 1.0, -5.0, -4.0]]",-143.0,-5.0,290.869263,-167.200882,9164.962891,6.914538,False,-5.0
452,453,0.1446,"[[-10.0, -8.0, 5.0, 3.0, -10.0]]",-298.0,-5.0,121.851273,-246.480621,9176.590820,6.939863,False,-5.0
453,454,0.1487,"[[-5.0, -0.0, -8.0, -2.0, -6.0]]",-129.0,-5.0,429.771454,-181.204361,9183.069336,6.882838,False,-5.0
454,455,0.1351,"[[-8.0, -2.0, -1.0, 2.0, 2.0]]",-77.0,-5.0,112.183014,-115.301933,9180.605469,6.202306,False,-5.0
